<a href="https://colab.research.google.com/github/madukisp/ferramentas_sala/blob/main/Emails_Forms_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# # Instalar gspread e autenticar
# !pip install --upgrade gspread
# !pip install --upgrade gspread google-auth




In [6]:
import gspread
from google.auth.transport.requests import Request
from google.colab import auth
from google.auth import default
import smtplib
from email.message import EmailMessage
from email.utils import formataddr
from datetime import datetime
import time

# Bloco 1: Extrair a lista de e-mails das unidades em atraso

# Autenticar com Google
auth.authenticate_user()
creds, _ = default()

# Autorizar o cliente do gspread com as credenciais do Google
client = gspread.authorize(creds)

# Abrir a planilha pelo link compartilhado
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1xCcNLeWV8F8tsLyuQE0MbZ7ZSbh0Upufia5Wt5Dhqns/edit?resourcekey=&gid=372499938#gid=372499938"  # Substitua pelo seu link
spreadsheet = client.open_by_url(spreadsheet_url)

# Abrir a aba de "Conferência"
worksheet = spreadsheet.worksheet("Conferência")

# Ler os dados da planilha
dados = worksheet.get_all_values()

# Filtrar e-mails das unidades que não responderam (coluna D deve conter '* Sem Resposta')
emails_para_enviar = []
for row in dados[7:]:  # Começar após o cabeçalho
    status_resposta = row[3]  # Coluna D (index 3)
    if status_resposta == "* Sem Resposta":
        # Colunas E, F, G contêm os e-mails dos contatos
        emails_gerente = row[4].split(',')
        emails_rt = row[5].split(',')
        emails_unidade = row[6].split(',')

        # Adicionar todos os e-mails à lista
        emails_para_enviar.extend(emails_gerente + emails_rt + emails_unidade)

# Remover duplicatas
emails_para_enviar = list(set(emails_para_enviar))

# Mostrar os e-mails extraídos
print("Emails a serem notificados:")
for email in emails_para_enviar:
    print(email)

Emails a serem notificados:

adlemos@saudedafamilia.org
ubs.autodromo@saudedafamilia.org
ubsjardimicaraiquintana@saudedafamilia.org
ematos@saudedafamilia.org
capsadulto3.grajau@saudedafamilia.org
eparanzini@saudedafamilia.org
pnetto@saudedafamilia.org
fsouza@saudedafamilia.org
ubsjardimtrescoracoes@saudedafamilia.org
clelima@saudedafamilia.org
ccardoso@saudedafamilia.org
apujol@saudedafamilia.org
lacarvalho@saudedafamilia.org
ybrasileiro@saudedafamilia.org 
tmota@saudedafamilia.org
ubs.parelheiros@saudedafamilia.org
capsadultocapela@saudedafamilia.org
bguedes@saudedafamilia.org
fabreu@saudedafamilia.org
ubs.shangrilaellus@saudedafamilia.org
vsato@saudedafamilia.org
ubsrecantocampobelo@saudedafamilia.org
emad.uad.capeladosocorro@saudedafamilia.org
mondei@saudedafamilia.org
afonseca@saudedafamilia.org
amagdaleno@saudedafamilia.org
lursilva@saudedafamilia.org
melsi@saudedafamilia.org
lrodrigues@saudedafamilia.org
vfelippe@saudedafamilia.org
eleal@saudedafamilia.org
elaisantos@saudedafamil

In [7]:
# Bloco 2: Preparar o envio das mensagens para as unidades em atraso

# Configurar email e senha
EMAIL_ADDRESS = 'asfsul.naoresponda@saudedafamilia.org'
EMAIL_PASSWORD = 'zsibjnptljtiduai'
DISPLAY_NAME = 'ASF SUL Não Responda'

# Função para obter o nome do mês
def get_month_name(month):
    months = [
        'janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho',
        'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro'
    ]
    return months[month - 1]

# Obter a data atual formatada
data_atual = datetime.now()
data_formatada = f"{data_atual.day} de {get_month_name(data_atual.month)} de {data_atual.year}"

# Ler o conteúdo HTML do arquivo template
try:
    with open('template.html', 'r', encoding='utf-8') as file:
        html_content = file.read()
except FileNotFoundError:
    print("Erro: O arquivo 'template.html' não foi encontrado na pasta do script.")
    exit()
except Exception as e:
    print(f"Erro ao ler o arquivo 'template.html': {str(e)}")
    exit()

# Substituir a data no template
html_content = html_content.replace('{data_atual}', data_formatada)

# Enviar email
for contato in emails_para_enviar:
    # Criar email
    mail = EmailMessage()
    mail['From'] = formataddr((DISPLAY_NAME, EMAIL_ADDRESS))
    mail['To'] = contato
    mail['Subject'] = 'LEMBRETE | Monitoramento do Sistema SIGA'
    mail.set_content(html_content, subtype='html')

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
            smtp.ehlo()  # Identificação com o servidor
            smtp.starttls()  # Conexão segura
            smtp.ehlo()
            smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            smtp.send_message(mail)
        print(f"Email enviado com sucesso para {contato}!")
    except smtplib.SMTPAuthenticationError:
        print("Erro de autenticação. Verifique o email e a senha.")
    except smtplib.SMTPConnectError:
        print("Erro ao conectar ao servidor SMTP. Verifique sua conexão.")
    except Exception as e:
        print(f"Erro ao enviar o email para {contato}: {str(e)}")

Erro ao enviar o email para : {'': (555, b'5.5.2 Syntax error, cannot decode response. For more information, go to\n5.5.2  https://support.google.com/a/answer/3221692 and review RFC 5321\n5.5.2 specifications. d9443c01a7336-20d17f8b4e1sm29076025ad.49 - gsmtp')}
Email enviado com sucesso para adlemos@saudedafamilia.org!
Email enviado com sucesso para ubs.autodromo@saudedafamilia.org!
Email enviado com sucesso para ubsjardimicaraiquintana@saudedafamilia.org!
Email enviado com sucesso para ematos@saudedafamilia.org!
Email enviado com sucesso para capsadulto3.grajau@saudedafamilia.org!
Email enviado com sucesso para eparanzini@saudedafamilia.org!
Email enviado com sucesso para pnetto@saudedafamilia.org!
Email enviado com sucesso para fsouza@saudedafamilia.org!
Email enviado com sucesso para ubsjardimtrescoracoes@saudedafamilia.org!
Email enviado com sucesso para clelima@saudedafamilia.org!
Email enviado com sucesso para ccardoso@saudedafamilia.org!
Email enviado com sucesso para apujol@sau